# Pytorch testing

# Initialize Random Data

In [3]:
import torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

In [4]:
n = 1  # Example value for n
additional_features = 40  # Size of the additional input feature vector
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Example inputs
bitboards = torch.randint(0, 2, (1, 12, 8, 8)).to(device).float()  # 8x8xn input

metadata = torch.randn(1, additional_features).to(device)  # nx1 input
game_results = torch.randn(1, 3).to(device)

In [5]:
print(bitboards)

tensor([[[[0., 1., 1., 0., 0., 1., 1., 1.],
          [1., 1., 0., 1., 1., 0., 0., 1.],
          [1., 1., 1., 1., 1., 1., 0., 1.],
          [0., 1., 1., 1., 0., 1., 0., 0.],
          [1., 0., 1., 0., 1., 0., 1., 0.],
          [0., 0., 1., 1., 1., 0., 1., 0.],
          [0., 0., 0., 0., 1., 1., 1., 1.],
          [1., 1., 1., 0., 0., 1., 1., 1.]],

         [[0., 1., 1., 0., 0., 0., 1., 0.],
          [1., 0., 1., 1., 0., 1., 0., 1.],
          [1., 0., 1., 1., 1., 1., 0., 0.],
          [0., 0., 1., 0., 0., 0., 0., 0.],
          [1., 1., 1., 0., 1., 0., 1., 0.],
          [1., 0., 0., 0., 1., 0., 1., 1.],
          [1., 0., 0., 0., 0., 1., 0., 1.],
          [0., 0., 0., 1., 1., 1., 1., 1.]],

         [[1., 1., 0., 0., 1., 0., 0., 0.],
          [1., 1., 0., 1., 1., 0., 1., 0.],
          [0., 0., 0., 1., 0., 1., 1., 1.],
          [1., 1., 1., 1., 0., 0., 0., 1.],
          [1., 1., 1., 1., 0., 1., 1., 0.],
          [1., 0., 0., 0., 0., 0., 0., 0.],
          [1., 0., 1., 1., 1

In [6]:
class BaseModel(nn.Module):
    def __init__(self, input_planes, additional_features=40):
        super(BaseModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=input_planes, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        
        self.fc_additional = nn.Linear(additional_features, 64 * 8 * 8)
        
        self.fc1 = nn.Linear(256 * 8 * 8, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 1)
    
    def forward(self, x, additional_input):
        x = F.relu(self.conv1(x))
        
        additional_input = F.relu(self.fc_additional(additional_input))
        additional_input = additional_input.view(-1, 64, 8, 8)
        
        x = torch.cat((x, additional_input), dim=1)
        
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [8]:
models = {
    "BaseModel": BaseModel(n, additional_features).to(device)
    # "AdditionalConvModel": AdditionalConvModel(n, additional_features).to(device),
    # "DifferentFCModel": DifferentFCModel(n, additional_features).to(device)
}


# Evaluate each model

model = BaseModel(n, additional_features).to(device)
print(f"Evaluating {model_name}")
output = model(bitboards, metadata)
print(output)

Evaluating BaseModel


RuntimeError: Given groups=1, weight of size [64, 1, 3, 3], expected input[1, 12, 8, 8] to have 1 channels, but got 12 channels instead

In [17]:
class CnnBaseModel(nn.Module):
    def __init__(self, input_planes, output_classes=3):
        super(CnnBaseModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=input_planes, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        
        self.fc1 = nn.Linear(256 * 8 * 8, 1024)
        self.fc2 = nn.Linear(1024, output_classes)


        
    def forward(self, x):
        print(f"After start shape: {x.shape}")
        x = F.relu(self.conv1(x))
        print(f"After conv1: {x.shape}")

        additional_input = F.relu(self.fc_additional(additional_input))
        additional_input = additional_input.view(-1, 64, 8, 8)
        
        x = torch.cat((x, additional_input), dim=1)
        
        x = F.relu(self.conv2(x))
        print(f"After conv2: {x.shape}")
        
        x = F.relu(self.conv3(x))
        print(f"After conv3: {x.shape}")
        
        x = x.view(x.size(0), -1)  # Flatten the tensor
        print(f"After flatten: {x.shape}")
        
        x = F.relu(self.fc1(x))
        print(f"After fc1: {x.shape}")
        
        x = self.fc2(x)
        print(f"After fc2: {x.shape}")
        
        x = F.log_softmax(x, dim=1)  # Softmax output
        print(f"After softmax: {x.shape}")
        
        return x

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_planes = 12  # Number of feature planes (e.g., 6 piece types for each player)
output_classes = 3  # Number of output classes for softmax

model = CnnBaseModel(input_planes, output_classes).to(device)

output = model(bitboards)
print(output)

After start shape: torch.Size([1, 12, 8, 8])
After conv1: torch.Size([1, 64, 8, 8])
After conv2: torch.Size([1, 128, 8, 8])
After conv3: torch.Size([1, 256, 8, 8])
After flatten: torch.Size([1, 16384])
After fc1: torch.Size([1, 1024])
After fc2: torch.Size([1, 3])
After softmax: torch.Size([1, 3])
tensor([[-1.0967, -1.0963, -1.1029]], grad_fn=<LogSoftmaxBackward0>)


In [ ]:
class FullModel(nn.Module):
    def __init__(self, input_planes, output_classes=3):
        super(FullModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=input_planes, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        
        self.fc1 = nn.Linear(256 * 8 * 8, 1024)
        self.fc2 = nn.Linear(1024, output_classes)


        self.fc_additional = nn.Linear(additional_features, 64 * 8 * 8)
        
    def forward(self, x):
        print(f"After start shape: {x.shape}")
        x = F.relu(self.conv1(x))
        print(f"After conv1: {x.shape}")

        x = self.fc_additional()
        
        x = F.relu(self.conv2(x))
        print(f"After conv2: {x.shape}")
        
        x = F.relu(self.conv3(x))
        print(f"After conv3: {x.shape}")
        
        x = x.view(x.size(0), -1)  # Flatten the tensor
        print(f"After flatten: {x.shape}")
        
        x = F.relu(self.fc1(x))
        print(f"After fc1: {x.shape}")
        
        x = self.fc2(x)
        print(f"After fc2: {x.shape}")
        
        x = F.log_softmax(x, dim=1)  # Softmax output
        print(f"After softmax: {x.shape}")
        
        return x